This notebook uses compiled version of the computeVPB function. To compile it run the command

    python setup.py build_ext --inplace

from the `python/` directory

See https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html for some basic tutorial

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import ripser
import persim
import importlib


In [25]:
import TDAvec
from TDAvec import pmin, pmax, DiagToPD, \
    computeVPB, computePL, computePS, computeNL, computeVAB, computeECC, computePES, computePI,\
    computeVPB_dim0, computeVPB_dim1
    

In [13]:
importlib.reload(TDAvec)


<module 'TDAvec' from '/Users/alekseiluchinsky/Work/STUDY/Thesis/VPB_Python/python/TDAvec.cpython-311-darwin.so'>

In [14]:
X = np.loadtxt("../R/unitCircle.csv", skiprows=1, delimiter=",")
D = ripser.ripser(X, thresh=2)["dgms"]
D[0][-1, 1] = 2


In [15]:
importlib.reload(TDAvec);

## computeVPB

In [47]:
vpb0

array([0.80900108, 3.64481538, 5.48661209, 6.38281627, 1.0139525 ])

In [46]:
PD = DiagToPD(D)
ySeqH0 = np.quantile(PD[0][:,1], np.arange(0, 1.1, 0.2))
vpb0 = computeVPB(PD, homDim=0, xSeq = [], ySeq = ySeqH0)
print(np.allclose(
    vpb0, 
    np.loadtxt("../R/vpb_0.csv", skiprows=1)
    , atol=1e-7))


xSeqH1 = np.quantile(PD[1][:,0], np.arange(0, 1.1, 0.2))
ySeqH1 = np.quantile(PD[1][:,1], np.arange(0, 1.1, 0.2))

vpb1 = computeVPB(PD, homDim = 1, xSeq=xSeqH1, ySeq=ySeqH1)
vpb1 = np.transpose(vpb1).reshape( (25,))
print(np.allclose(
    vpb1, 
    np.loadtxt("../R/vpb_1.csv", skiprows=1)
    , atol=1e-7))


True
True


In [64]:
ySeqH1

array([0.00168392, 0.00642258, 0.02016019, 0.02807069, 0.05783653,
       0.86032665])

In [51]:
vpb1

array([0.        , 0.00049954, 0.00903382, 0.03050973, 0.00579736,
       0.00951945, 0.02610533, 0.02036727, 0.00428711, 0.0107446 ,
       0.04257685, 0.10534681, 0.14588074, 0.        , 0.        ,
       0.03280356, 0.02767857, 0.10890892, 0.13188222, 0.01683049,
       0.29393937, 0.3162986 , 0.3344511 , 0.35674869, 0.36362269])

In [50]:
computeVPB(PD, homDim = 1, xSeq=xSeqH1, ySeq=ySeqH1)

array([[0.        , 0.00951945, 0.04257685, 0.03280356, 0.29393937],
       [0.00049954, 0.02610533, 0.10534681, 0.02767857, 0.3162986 ],
       [0.00903382, 0.02036727, 0.14588074, 0.10890892, 0.3344511 ],
       [0.03050973, 0.00428711, 0.        , 0.13188222, 0.35674869],
       [0.00579736, 0.0107446 , 0.        , 0.01683049, 0.36362269]])

In [37]:
vpb0

array([0.80900108, 3.64481538, 5.48661209, 6.38281627, 1.0139525 ])

In [36]:
x = PD[0][:,0]
y = PD[0][:,1]
tau = 0.3
np.allclose(
    computeVPB_dim0(x=PD[0][0,:], y=PD[0][:,1], ySeq = ySeqH0, lam = tau*y),\
    np.loadtxt("../R/vpb_0.csv", skiprows=1)
)

True

In [22]:
computeVPB

<cyfunction computeVPB at 0x1380f5ff0>

## computePI

In [7]:
resB, resP = 5, 5
#
minPH0, maxPH0 = np.min(PD[0][:,1]), np.max(PD[0][:,1])
ySeqH0 = np.linspace(minPH0, maxPH0, resP+1)
xSeqH0 = np.zeros( resB+1)

minBH1, maxBH1 = np.min(PD[1][:,0]), np.max(PD[1][:,0])
xSeqH1 = np.linspace(minBH1, maxBH1, resB+1)
minPH1, maxPH1 = np.min(PD[1][:,1]), np.max(PD[1][:,1])
ySeqH1 = np.linspace(minPH1, maxPH1, resP+1)

sigma = 0.5*(maxPH0-minPH0)/resP
pi0 = computePI(PD, homDim = 0, xSeq = xSeqH0, ySeq = ySeqH0, sigma = sigma)
pi0_R = np.loadtxt("../R/pi_0.csv", skiprows=1)
print(0, np.allclose(pi0, pi0_R))

sigma = 0.5*(maxPH1-minPH1)/resP
pi1 = computePI(PD, homDim = 1, xSeq = xSeqH1, ySeq = ySeqH1, sigma = sigma)
pi1_R = np.loadtxt("../R/pi_1.csv", skiprows=1)
print(1, np.allclose(pi1, pi1_R))


0 True
1 True


## Others

In [8]:
scaleSeq = np.linspace(0, 2, 11)
# universal function for comparison
def compareResults(func, R_prefix, maxDim = 1, D=D, scaleSeq = scaleSeq, atol = 1e-7):
    print(f"Comparing compute{R_prefix.upper()}:")
    # for each of the dims
    for d in range(maxDim+1):
        # calc python result
        pyth = func(D, d, scaleSeq)
        # read R result
        R = np.loadtxt(f"../R/{R_prefix}_{d}.csv", skiprows=1)
        # report if they are equal
        cond = np.allclose(pyth, R, atol=atol)
        print("\t dim=", d, ":", cond)
        assert cond
        


In [9]:
testDict = {"pl":computePL, "ps":computePS, "nl":computeNL, "vab":computeVAB, "ecc":computeECC, "pes": computePES}
for p in testDict.keys():
    compareResults(testDict[p], p)

Comparing computePL:
	 dim= 0 : True
	 dim= 1 : True
Comparing computePS:
	 dim= 0 : True
	 dim= 1 : True
Comparing computeNL:
	 dim= 0 : True
	 dim= 1 : True
Comparing computeVAB:
	 dim= 0 : True
	 dim= 1 : True
Comparing computeECC:
	 dim= 0 : True
	 dim= 1 : True
Comparing computePES:
	 dim= 0 : True
	 dim= 1 : True
